In [ ]:
%env OPENAI_API_KEY=keyhere

In [ ]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

import asyncio
from kagent.tools.docs import search_documentation

model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
)

cncf_docs_agent = AssistantAgent(
    "kubernetes_docs_agent",
    description="This agent allows you to get data from the CNCF docs databases.",
    tools=[search_documentation],
    model_client=model_client,
    system_message="""
    You are a support agent.

    You have access to several tools:
    - 'searchDocumentation' to search in the documentation.

    Execute all the following steps:
    1. Product identification
    - Check if you know what product the question is about (Only supported products are: 'kubernetes', 'istio', 'argo')
    - If it's not the case, ask what the product is

    2. Information Gathering
    - Search in the documentation for information related to the question the user has submitted

    3. Answer the question
    - Use all the information you gathered to provide a valuable answer to the user
    - Provide links to the documentation whenever possible`,
    """,
)

user_proxy = UserProxyAgent("user_proxy", input_func=input)

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination


team = RoundRobinGroupChat(
    [
        cncf_docs_agent,
    ],
    termination_condition=termination,
)

task = "How do I do a canary upgrade of a pod in kubernetes using argo rollouts?"

# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- user ----------
How do I do a canary upgrade of a pod in kubernetes using argo rollouts?
---------- kubernetes_docs_agent ----------
[FunctionCall(id='call_Gvx7UU6IuTxjouUpjXWQpJZc', arguments='{"query": "canary upgrade pod", "product": "kubernetes", "version": null, "limit": 5}', name='search_documentation'), FunctionCall(id='call_JyimKgtekwI6MDFDocCcqVqy', arguments='{"query": "canary upgrade pod", "product": "argo", "version": null, "limit": 5}', name='search_documentation')]


ERROR:root:Downloading database for argo from https://doc-sqlite-db.s3.sa-east-1.amazonaws.com/argo.db
ERROR:root:Successfully downloaded database for argo


---------- kubernetes_docs_agent ----------
[FunctionExecutionResult(content='[{"distance": 1.0185943841934204, "content": "kubectl attach mypod\\n"}, {"distance": 1.0381801128387451, "content": "kubectl attach mypod -c ruby-container\\n"}, {"distance": 1.0388245582580566, "content": " refused because that would leave only one pod available for the deployment.\\n\\nThe deployment creates a replacement for `pod-b` called `pod-e`. Because there are not enough resources in the cluster to schedule `pod-e` the drain will again block. The cluster may end up in this state:\\n\\n| node-1 _drained_ | node-2 | node-3 | _no node_ |\\n|--- | --- | --- | ---|\\n| | pod-b _terminating_ | pod-c _available_ | pod-e _pending_ |\\n| | pod-d _available_ | pod-y | |\\n\\nAt this point, the cluster administrator needs to add a node back to the cluster to proceed with the upgrade.\\n\\nYou can see how Kubernetes varies the rate at which disruptions can happen, according to:\\n\\n*   how many replicas an app

In [ ]:
print(team.dump_component().model_dump_json(indent=2))